In [1]:
!pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install requests


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
import json
from datetime import datetime, timedelta
API_KEY=""
lat=29.3956
lon=71.6836

In [4]:
yesterday=int((datetime.utcnow()-timedelta(days=1)).timestamp())
yesterday

C:\Users\PMLS\AppData\Local\Temp\ipykernel_24740\1189914141.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  yesterday=int((datetime.utcnow()-timedelta(days=1)).timestamp())


1767372320

In [5]:
start=yesterday- 86400
end=yesterday
# trying to fetch data for a single point in time
weather_url = (
    f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
)
#^ fetching current weather data for bwp
# free tier of openweather doesnot include historical weather data
# switching to open meteo/openAQ or Waqi (will decide later)
weather_response=requests.get(weather_url)
weather_response.status_code

200

In [6]:
weather_response

<Response [200]>

In [7]:
weather_data=weather_response.json()
weather_data.keys()

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])

In [8]:
weather_data

{'coord': {'lon': 71.6836, 'lat': 29.3956},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 13.36,
  'feels_like': 11.77,
  'temp_min': 13.36,
  'temp_max': 13.36,
  'pressure': 1018,
  'humidity': 39,
  'sea_level': 1018,
  'grnd_level': 1005},
 'visibility': 10000,
 'wind': {'speed': 2.74, 'deg': 195, 'gust': 2.74},
 'clouds': {'all': 0},
 'dt': 1767476722,
 'sys': {'country': 'PK', 'sunrise': 1767492496, 'sunset': 1767529647},
 'timezone': 18000,
 'id': 1183880,
 'name': 'Bahawalpur',
 'cod': 200}

In [9]:
pollution_url=f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={API_KEY}"

In [10]:
!pip install openaq


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from openaq import OpenAQ

In [12]:
now= datetime.utcnow()

C:\Users\PMLS\AppData\Local\Temp\ipykernel_24740\3266932151.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now= datetime.utcnow()


In [13]:
twoMonthsAgo= now-timedelta(days=60)
dateFrom= twoMonthsAgo.isoformat() + "Z"
dateTo= now.isoformat()+"Z"

# printig start and end dates:
print("Fetching data from: ",dateFrom)
print("Fetching till: ",dateTo)

Fetching data from:  2025-11-04T21:45:35.042953Z
Fetching till:  2026-01-03T21:45:35.042953Z


In [ ]:
results=[]
baseURL="https://api.openaq.org/v3/measurements"
parameters=["pm25","pm10","no2","so2","co","o3"]
token = ""
headers = {"Authorization": f"Bearer {token}"}
page=1
while True:
    url = (
        f"{baseURL}?coordinates={lat},{lon}"
        f"&radius=50000"
        f"&date_from={dateFrom}&date_to={dateTo}"
        f"&parameter={','.join(parameters)}"
        f"&limit=100&page={page}"
    )
    response=requests.get(url,headers=headers)
    if response.status_code!=200:
        print("Error: ",response.status_code)
        break

    data=response.json()
    result=data.get("results",[])
    if not result:
        break
    results.extend(result)    
    print(f"Fetched page:{page}, records so far: {len(results)}")
    page+=1
print(f"Total records fetched: {len(results)}")

Error:  401
Total records fetched: 0


In [ ]:
# lat, lon = 29.3956, 71.6836
# dateFrom = "2025-11-01T00:00:00Z"
# dateTo   = "2025-12-31T23:59:59Z"

# url = (
#     f"https://api.openaq.org/v3/measurements"
#     f"?latitude={lat}&longitude={lon}"
#     f"&radius=50000"
#     f"&date_from={dateFrom}&date_to={dateTo}"
#     f"&parameter=pm25,pm10"
#     f"&limit=5"
# )

# token = ""
# headers = {"X-API-Key": ""}

# response = requests.get(url, headers=headers)
# print(response.status_code)
# print(response.text)

404
{"detail":"Not Found"}


In [ ]:
# station_url = f"https://api.openaq.org/v3/locations?latitude={lat}&longitude={lon}&radius=100000"
# token=""
# headers = {"X-API-Key": token}
# response = requests.get(station_url, headers=headers)
# print(response)
# # stations = response.json().get("results", [])
# # print(stations)

<Response [422]>


In [17]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import openmeteo_requests

import pandas as pd

# Setup the Open-Meteo API client with cache and retry on error
openmeteo = openmeteo_requests.Client()

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 29.3978,
	"longitude": 71.6752,
	"hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "pressure_msl", "wind_direction_10m", "precipitation", "cloud_cover_low"],
	"timezone": "GMT",
	"past_days": 61,
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(5).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"timestamp": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low

weather_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", weather_dataframe)


Coordinates: 29.375°N 71.625°E
Elevation: 121.0 m asl
Timezone: NoneNone
Timezone difference to GMT+0: 0s

Hourly data
                      timestamp  temperature_2m  relative_humidity_2m  \
0    2025-11-03 00:00:00+00:00       16.734999                  75.0   
1    2025-11-03 01:00:00+00:00       16.584999                  75.0   
2    2025-11-03 02:00:00+00:00       16.635000                  77.0   
3    2025-11-03 03:00:00+00:00       19.135000                  76.0   
4    2025-11-03 04:00:00+00:00       22.834999                  67.0   
...                        ...             ...                   ...   
1627 2026-01-09 19:00:00+00:00        8.435000                  82.0   
1628 2026-01-09 20:00:00+00:00        8.185000                  81.0   
1629 2026-01-09 21:00:00+00:00        7.935000                  81.0   
1630 2026-01-09 22:00:00+00:00        7.635000                  81.0   
1631 2026-01-09 23:00:00+00:00        7.385000                  82.0   

      wind_spee

In [19]:
openmeteo = openmeteo_requests.Client()

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 29.3978,
	"longitude": 71.6752,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "carbon_dioxide"],
    "past_days": 61,
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ozone = hourly.Variables(5).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"timestamp": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide

pollutants_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", pollutants_dataframe)


Coordinates: 29.400001525878906°N 71.69999694824219°E
Elevation: 121.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                      timestamp        pm10       pm2_5  carbon_monoxide  \
0    2025-11-03 00:00:00+00:00   51.099998   48.500000            342.0   
1    2025-11-03 01:00:00+00:00   46.799999   44.599998            466.0   
2    2025-11-03 02:00:00+00:00   45.299999   43.200001            632.0   
3    2025-11-03 03:00:00+00:00   42.700001   40.900002            729.0   
4    2025-11-03 04:00:00+00:00   42.500000   40.599998            695.0   
...                        ...         ...         ...              ...   
1579 2026-01-07 19:00:00+00:00  101.000000   96.599998           1796.0   
1580 2026-01-07 20:00:00+00:00  104.500000   99.900002           1713.0   
1581 2026-01-07 21:00:00+00:00  111.300003  105.500000           1643.0   
1582 2026-01-07 22:00:00+00:00  118.699997  112.500000           1609.0   
1583 2026-01-07 23:00:00+00:00  124.800003  118.300

In [20]:
len(weather_dataframe)

1632

In [21]:
len(pollutants_dataframe)

1584

In [22]:
print("Weather Ranges: ")
print(f"weather data starting from: {weather_dataframe["timestamp"].min()}")
print(f"weather data ending on: {weather_dataframe["timestamp"].max()}")

Weather Ranges: 
weather data starting from: 2025-11-03 00:00:00+00:00
weather data ending on: 2026-01-09 23:00:00+00:00


In [23]:
print(f"pollutants data starting from: {pollutants_dataframe["timestamp"].min()}")
print(f"pollutants data ending on: {pollutants_dataframe["timestamp"].max()}")

pollutants data starting from: 2025-11-03 00:00:00+00:00
pollutants data ending on: 2026-01-07 23:00:00+00:00


In [24]:
!pip install pyarrow


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
weather_dataframe.rename(columns={"date": "timestamp"}, inplace=True)
pollutants_dataframe.rename(columns={"date": "timestamp"}, inplace=True)

In [26]:
weather_dataframe.head()

,timestamp,temperature_2m,relative_humidity_2m,wind_speed_10m,pressure_msl,wind_direction_10m,precipitation,cloud_cover_low
0,2025-11-03 00:00:00+00:00,16.734999,75.0,5.760000,1008.500000,180.000000,0.0,0.0
1,2025-11-03 01:00:00+00:00,16.584999,75.0,5.400000,1008.900024,180.000000,0.0,0.0
2,2025-11-03 02:00:00+00:00,16.635000,77.0,5.052841,1009.200012,184.085541,0.0,0.0
3,2025-11-03 03:00:00+00:00,19.135000,76.0,3.415260,1009.700012,198.435043,0.0,0.0
4,2025-11-03 04:00:00+00:00,22.834999,67.0,3.600000,1010.299988,233.130020,0.0,0.0


In [27]:
print(weather_dataframe.columns)

Index(['timestamp', 'temperature_2m', 'relative_humidity_2m', 'wind_speed_10m',
       'pressure_msl', 'wind_direction_10m', 'precipitation',
       'cloud_cover_low'],
      dtype='object')


In [28]:
weather_dataframe.to_parquet(
    "data/raw/weather/weather_bwp_last2months.parquet",
    engine="pyarrow",
    index=False
)

OSError: Cannot save file into a non-existent directory: 'data\raw\weather'

In [ ]:
weather_dataframe.to_csv("weather_bwp.csv", index=False)
pollutants_dataframe.to_csv("pollutants_bwp.csv", index=False)

In [29]:
weather_dataframe.to_parquet("weather_bwp.parquet",index=False)